In [1]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 52.9 MB/s eta 0:00:00


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.2 MB/s eta 0:00:00


In [3]:
# Import nltk libraries and download
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

import re
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [5]:
# Download the train.csv from the google drive by providing its Unique ID
! gdown 1hdJ8HxCOeS7ZCtCGPLNfKoWt4h_8qJUV

Downloading...
From: https://drive.google.com/uc?id=1hdJ8HxCOeS7ZCtCGPLNfKoWt4h_8qJUV
To: /content/train.csv
100% 68.8M/68.8M [00:02<00:00, 29.8MB/s]


In [6]:
df = pd.read_csv("/content/train.csv")

In [7]:
df.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
df.groupby('toxic').describe()

severe_toxic                                                obscene  \
             count      mean       std  min  25%  50%  75%  max     count   
toxic                                                                       
0         144277.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  144277.0   
1          15294.0  0.104289  0.305645  0.0  0.0  0.0  0.0  1.0   15294.0   

                 ... insult      identity_hate                                \
           mean  ...    75%  max         count      mean       std  min  25%   
toxic            ...                                                           
0      0.003625  ...    0.0  1.0      144277.0  0.000714  0.026710  0.0  0.0   
1      0.518242  ...    1.0  1.0       15294.0  0.085131  0.279086  0.0  0.0   

                      
       50%  75%  max  
toxic                 
0      0.0  0.0  1.0  
1      0.0  0.0  1.0  

[2 rows x 40 columns]

In [9]:
len(df)

159571

In [10]:
# Split the dataset into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(df['comment_text'],df['toxic'], test_size=0.2, random_state=42)

In [11]:
X_train.head()

140030    Grandma Terri Should Burn in Trash \nGrandma T...
159124    , 9 May 2009 (UTC)\nIt would be easiest if you...
60006     "\n\nThe Objectivity of this Discussion is dou...
65432                 Shelly Shock\nShelly Shock is. . .( )
154979    I do not care. Refer to Ong Teng Cheong talk p...
Name: comment_text, dtype: object

In [12]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [13]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [14]:
lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()
eng_stopwords = set(stopwords.words("english"))

In [15]:
# Create a dictionary of Aphost lookup
APHOST_LOOKUP = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"
}

In [16]:
# Define the function for data cleaning
def clean_data_pipeline(comment):
    comment=comment.lower() # Convert to lower case
    comment=re.sub("\\n","",comment) # Remove new line
    # Remove the leaky elements like ip,user
    comment=re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}","",comment)
    comment=re.sub("\[\[.*\]","",comment) # Remove users
    words=tokenizer.tokenize(comment) # Tokenize into words
    # Replace apostrophe with reference to the Aphost Dictionary 
    words=[APHOST_LOOKUP[word] if word in APHOST_LOOKUP else word for word in words]
    words=[lem.lemmatize(word, "v") for word in words] # Lemmatization
    words = [w for w in words if not w in eng_stopwords] # Remove stopwords
    clean_sent=" ".join(words)
    return(clean_sent)

#### Build Model

In [17]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [18]:
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [19]:
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [21]:
len(X_train)

127656

In [22]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

#### Data Cleaning

In [23]:
# Clean the data
X_train_cleaned = X_train.apply(clean_data_pipeline)

#### Train the Model

In [24]:
# Train the model
model.fit(X_train_cleaned, y_train, validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
3192/3192 [==============================] - 1264s 392ms/step - loss: 0.2211 - accuracy: 0.9198 - val_loss: 0.1834 - val_accuracy: 0.9312
Epoch 2/5
3192/3192 [==============================] - 1251s 392ms/step - loss: 0.1880 - accuracy: 0.9306 - val_loss: 0.1748 - val_accuracy: 0.9319
Epoch 3/5
3192/3192 [==============================] - 1250s 392ms/step - loss: 0.1817 - accuracy: 0.9325 - val_loss: 0.1679 - val_accuracy: 0.9357
Epoch 4/5
3192/3192 [==============================] - 1250s 392ms/step - loss: 0.1806 - accuracy: 0.9330 - val_loss: 0.1655 - val_accuracy: 0.9368
Epoch 5/5
3192/3192 [==============================] - 1250s 391ms/step - loss: 0.1793 - accuracy: 0.9336 - val_loss: 0.1649 - val_accuracy: 0.9380


In [25]:
model.evaluate(X_test, y_test)

998/998 [==============================] - 314s 314ms/step - loss: 0.1651 - accuracy: 0.9376


[0.16514627635478973, 0.9375528693199158]

In [ ]:
# Evaluate the model on the testing data
from sklearn.metrics import accuracy_score,  hamming_loss
y_pred = model.predict(X_test)

In [28]:
import numpy as np

In [32]:
pred = np.argmax(y_pred, axis=1)
acc = accuracy_score(X_test, pred)
print("The accuracy: {:.2f}%".format(acc*100))
hl = hamming_loss(X_test, pred)
print('Hamming loss is {}'.format(hl))

The accuracy: 0.00%
Hamming loss is 1.0
